In [7]:
# from mask_former import mask_former_model
from detectron2.config import get_cfg

In [8]:
# Create configs and perform basic setups

cfg = get_cfg()
# add_deeplab_config(cfg)
# add_mask_former_config(cfg)
cfg.set_new_allowed(True)
# cfg.merge_from_file("MaskFormer/configs/custom/MaskAVSL_swin_large.yaml")
# cfg.merge_from_list(args.opts)
cfg.MODEL.DEVICE = "cuda:1"
cfg.eval_only = True
cfg.freeze()
# default_setup(cfg, args)
# Setup logger for "mask_former" module
# setup_logger(output=cfg.OUTPUT_DIR, distributed_rank=comm.get_rank(), name="mask2former")

In [ ]:
from mask_former.modeling.backbone import swin

backbone = swin.D2SwinTransformer(cfg=cfg, input_shape=(256,256,3))

In [ ]:
mf = mask_former_model.MaskFormer()

In [14]:
from detectron2.layers import ShapeSpec
from MODULES.MaskFormer.modeling.backbone.swin import D2SwinTransformer
from MODULES.MaskFormer.modeling.pixel_decoder.pixel_decoder import TransformerEncoderPixelDecoder
from MODULES.MaskFormer.modeling.transformer.mask_predictor import MaskPredictor

class AVSLmodel():
    def build_swin_backbone(self):
        model = D2SwinTransformer(self.cfg, self.input_shape)
        model.init_weights(self.cfg.MODEL.WEIGHTS)
        return model
    
    def build_pixel_decoder(self):
        model = TransformerEncoderPixelDecoder(self.cfg, self.input_shape)
        return model
    
    def __init__(self, cfg):
        self.audiomodule_weight = {
            'vggish': 'MODULES/ckpt/vggish-10086976.pth',
            'pca': 'MODULES/ckpt/vggish_pca_params-970ea276.pth'}
        self.cfg = cfg
        self.input_shape = ShapeSpec(channels=len(self.cfg.MODEL.PIXEL_MEAN))
        self.backbone = self.build_swin_backbone()
        self.pixel_decoder = self.build_pixel_decoder()
        self.mask_predictor = MaskPredictor(self.cfg, self.cfg.MODEL.SEM_SEG_HEAD.CONVS_DIM, self.audiomodule_weight)


    def forward(self, x):
        image_features = self.backbone(x['images'].tensor)
        pp_embed, image_features = self.pixel_decoder(image_features)
        decoder_outputs = self.mask_predictor(image_features, pp_embed, x['audios'])

        out = decoder_outputs
        return out

In [1]:
model = AVSLmodel(cfg)

NameError: name 'AVSLmodel' is not defined

In [1]:
import torch

In [40]:
a = torch.tensor((((1,2,3),(3,4,4)),((4,5,5),(6,5,3))), dtype=torch.float32)
b = torch.tensor((((11,22,23),(33,34,34)),((34,35,5),(6,5,3))), dtype=torch.float32)




In [18]:
torch.mean(a,dim=-1)

tensor([2., 5.])

In [19]:
def gmean(input_x, dim):
    log_x = torch.log(input_x)
    return torch.exp(torch.mean(log_x, dim=dim))

In [21]:
gmean(a,dim=-1)

tensor([1.8171, 4.9324])

In [22]:
def asd(x):
    return x+2

In [26]:
a * a

tensor([[ 1.,  4.,  9.],
        [16., 25., 36.]])

In [28]:
from numpy import dot
from numpy.linalg import norm

def cos_sim(a,b):
    return dot(a,b)/(norm(a) * norm(b))

In [29]:
cos_sim(a,a)

ValueError: shapes (2,3) and (2,3) not aligned: 3 (dim 1) != 2 (dim 0)

In [54]:
cos = torch.nn.CosineSimilarity(dim=-1)

In [55]:
cos(a,b)

tensor([[0.9841, 0.9935],
        [0.8432, 1.0000]])